In [ ]:
%pip install chromedriver-autoinstaller

In [ ]:
%pip install selenium

In [ ]:
%pip install pandas

In [ ]:
%pip install 'requests'

In [ ]:
# Installe automatiquement le Chromedriver approprié pour le système
chromedriver_autoinstaller.install
#ou chromedriver_autoinstaller.install()

In [59]:
import chromedriver_autoinstaller
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import json
from datetime import datetime

## Étape 1: Scrapping du site de l'APEC

## Data ingénieur

Normalement y'a juste à faire tourner le code, ça va ouvrir automatiquement le site de l'APEC sur ton navigateur (c'est config pour Chrome).  
Ça met environ 1h à tourner (ça dépend probablement de la qualité du wifi).  
Tu devrais recup entre 2000 et 3000 annonces.

In [ ]:

# Utiliser Selenium pour charger la page web
url = "https://www.apec.fr/candidat/recherche-emploi.html/emploi?motsCles=ing%C3%A9nieur%20Data&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&page=0"
driver = webdriver.Chrome()
driver.get(url)

# Attendre que les éléments des offres d'emploi soient chargés
driver.implicitly_wait(10)  # Attendre jusqu'à 10 secondes au maximum

# Cliquer sur le bouton de refus des cookies en utilisant une approche différente
try:
    # Attendre que le bouton soit cliquable
    reject_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-reject-all-handler"]'))
    )
    reject_button.click()
except Exception as e:
    print(f"Erreur lors du clic sur le bouton de refus des cookies : {e}")

# Initialiser des listes pour stocker les informations 
titles_list = []
description_list = []
profil_list = []
info_entreprise_list = []
type_contrat_list = []
lieu_list = []
date_publication_list = []
salaire_list = []
niveau_experience_list = []
statut_poste_list = []
secteur_activite_list = []
#langues_list = []
entreprise_list = []

# Boucle pour parcourir les pages
while True:
    # Récupérer le nombre d'offres
    offers_count = len(driver.find_elements(By.XPATH, '/html/body/main/div/apec-offres/div/apec-recherche-emploi/section/div[2]/div[3]/div/div/div[2]/div[2]/div/a/apec-recherche-resultat/div/div'))

    # Récupérer les titres, descriptions et profils de chaque offre
    for i in range(1, offers_count + 1):
        try:
            # Récupérer le nom de l'entreprise avant de cliquer sur l'offre
            entreprise_xpath = f'/html/body/main/div/apec-offres/div/apec-recherche-emploi/section/div[2]/div[3]/div/div/div[2]/div[2]/div[{i}]/a/apec-recherche-resultat/div/div/div/div[2]/p[1]'
            entreprise_element = driver.find_element(By.XPATH, entreprise_xpath)
            entreprise = entreprise_element.text
                
            lieu_xpath = f'/html/body/main/div/apec-offres/div/apec-recherche-emploi/section/div[2]/div[3]/div/div/div[2]/div[2]/div[{i}]/a/apec-recherche-resultat/div/div/div/div[2]/ul[2]/li[2]'
            lieu_element = driver.find_element(By.XPATH, lieu_xpath)
            lieu = lieu_element.text

            type_contrat_xpath = f'/html/body/main/div/apec-offres/div/apec-recherche-emploi/section/div[2]/div[3]/div/div/div[2]/div[2]/div[{i}]/a/apec-recherche-resultat/div/div/div/div[2]/ul[2]/li[1]'
            type_contrat_element = driver.find_element(By.XPATH, type_contrat_xpath)
            type_contrat = type_contrat_element.text

            date_publication_xpath = f'/html/body/main/div/apec-offres/div/apec-recherche-emploi/section/div[2]/div[3]/div/div/div[2]/div[2]/div[{i}]/a/apec-recherche-resultat/div/div/div/div[2]/ul[2]/li[3]'
            date_publication_element = driver.find_element(By.XPATH, date_publication_xpath)
            date_publication = date_publication_element.text

            # Cliquer sur l'offre
            offer_xpath = f'/html/body/main/div/apec-offres/div/apec-recherche-emploi/section/div[2]/div[3]/div/div/div[2]/div[2]/div[{i}]/a/apec-recherche-resultat/div/div'
            offer = driver.find_element(By.XPATH, offer_xpath)
            offer.click()

            # Attendre que les informations soient visibles
            title_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '/html/body/main/div/apec-offres/div/apec-detail-emploi/apec-header-nav/div/div/div/div/nav/div/div/h1'))
            )
            title = title_element.text

            description_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '/html/body/main/div/apec-offres/div/apec-detail-emploi/div/div/div[2]/article/div/div/apec-poste-informations/div/div[2]/div[1]/p[1]'))
            )
            description = description_element.text

            profil_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '/html/body/main/div/apec-offres/div/apec-detail-emploi/div/div/div[2]/article/div/div/apec-poste-informations/div/div[2]/div[1]/p[2]'))
            )
            profil = profil_element.text

            info_entreprise_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '/html/body/main/div/apec-offres/div/apec-detail-emploi/div/div/div[2]/article/div/div/apec-poste-informations/div/div[2]/div[1]/div[4]/p'))
            )
            info_entreprise = info_entreprise_element.text

            salaire_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '/html/body/main/div/apec-offres/div/apec-detail-emploi/div/div/div[2]/article/div/div/apec-poste-informations/div/div[1]/div[1]/span'))
            )
            salaire = salaire_element.text

            niveau_experience_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '/html/body/main/div/apec-offres/div/apec-detail-emploi/div/div/div[2]/article/div/div/apec-poste-informations/div/div[1]/div[3]/span'))
            )
            niveau_experience = niveau_experience_element.text

            statut_poste_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '/html/body/main/div/apec-offres/div/apec-detail-emploi/div/div/div[2]/article/div/div/apec-poste-informations/div/div[1]/div[5]/span'))
            )
            statut_poste = statut_poste_element.text

            secteur_activite_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '/html/body/main/div/apec-offres/div/apec-detail-emploi/div/div/div[2]/article/div/div/apec-poste-informations/div/div[1]/div[7]/span'))
            )
            secteur_activite = secteur_activite_element.text

            #langues_element = WebDriverWait(driver, 10).until(
            #    EC.visibility_of_element_located((By.XPATH, '/html/body/main/div/apec-offres/div/apec-detail-emploi/div/div/div[2]/article/div/div/apec-poste-informations/div/div[2]/div[1]/div[1]/div/div[2]/span/div/p/label'))
            #)
            #langues = langues_element.text

            # Ajouter les informations à leurs listes respectives
            titles_list.append(title)
            description_list.append(description)
            profil_list.append(profil)
            info_entreprise_list.append(info_entreprise)
            type_contrat_list.append(type_contrat)
            lieu_list.append(lieu)
            date_publication_list.append(date_publication)
            salaire_list.append(salaire)
            niveau_experience_list.append(niveau_experience)
            statut_poste_list.append(statut_poste)
            secteur_activite_list.append(secteur_activite)
            #langues_list.append(langues)
            entreprise_list.append(entreprise)

            # Revenir à la page précédente
            driver.back()

            # Attendre que la page précédente soit chargée
            driver.implicitly_wait(5)
        except Exception as e:
            print(f"Erreur lors du traitement de l'offre {i} : {e}")

    # Aller à la page suivante
    try:
        next_page_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/main/div/apec-offres/div/apec-recherche-emploi/section/div[2]/div[3]/div/div/div[2]/div[2]/div[21]/apec-recherche-pagination/nav/ul/li[8]/a'))
        )
        next_page_button.click()
    except Exception as e:
        print(f"Fin de la pagination : {e}")
        break

# Créer un DataFrame à partir des listes d'informations supplémentaires
df = pd.DataFrame({
    "Titres": titles_list, 
    "Descriptions": description_list, 
    "Profils": profil_list,
    "Info Entreprise": info_entreprise_list,
    "Type Contrat": type_contrat_list,
    "Lieu": lieu_list,
    "Date de publication": date_publication_list,
    "Salaire": salaire_list,
    "Niveau d'expérience": niveau_experience_list,
    "Statut du poste": statut_poste_list,
    "Secteur d'activité": secteur_activite_list,
    #"Langues": langues_list
    "Entreprise": entreprise_list
})

# Fermer le navigateur
driver.quit()


In [ ]:
# Filtrer les lignes contenant les mots clés dans la colonne "Titres" (environ 150 restants)
data_engineer_df = df[df['Titres'].str.contains(r'\b(data ingénieur|data ingenieur|data engineer)\b', case=False, na=False)]
data_engineer_df.shape

## Data scientist

In [ ]:
# URL de base
base_url = "https://www.apec.fr/candidat/recherche-emploi.html/emploi?motsCles=data%20scientist&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&typesConvention=143706&page="

# Configuration du navigateur
driver = webdriver.Chrome()

# Initialiser des listes pour stocker les informations
titles_list = []
description_list = []
profil_list = []
info_entreprise_list = []
type_contrat_list = []
lieu_list = []
date_publication_list = []
salaire_list = []
niveau_experience_list = []
statut_poste_list = []
secteur_activite_list = []
entreprise_list = []

# Boucle sur les 4 premières pages
for page in range(1, 5):
    # Construire l'URL de la page actuelle
    url = base_url + str(page)
    
    # Charger l'URL
    driver.get(url)
    
    # Attendre que les éléments des offres d'emploi soient chargés
    driver.implicitly_wait(10)  # Attendre jusqu'à 10 secondes au maximum

    # Cliquer sur le bouton de refus des cookies en utilisant une approche différente
    try:
        # Attendre que le bouton soit cliquable
        reject_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-reject-all-handler"]'))
        )
        reject_button.click()
    except Exception as e:
        print(f"Erreur lors du clic sur le bouton de refus des cookies : {e}")

    # Récupérer le nombre d'offres sur la page actuelle
    offers_count = len(driver.find_elements(By.XPATH, '/html/body/main/div/apec-offres/div/apec-recherche-emploi/section/div[2]/div[3]/div/div/div[2]/div[2]/div/a/apec-recherche-resultat/div/div'))

    # Récupérer les titres, descriptions et profils de chaque offre sur la page actuelle
    for i in range(1, offers_count + 1):
        try:
            # Récupérer le nom de l'entreprise avant de cliquer sur l'offre
            entreprise_xpath = f'/html/body/main/div/apec-offres/div/apec-recherche-emploi/section/div[2]/div[3]/div/div/div[2]/div[2]/div[{i}]/a/apec-recherche-resultat/div/div/div/div[2]/p[1]'
            entreprise_element = driver.find_element(By.XPATH, entreprise_xpath)
            entreprise = entreprise_element.text
            
            lieu_xpath = f'/html/body/main/div/apec-offres/div/apec-recherche-emploi/section/div[2]/div[3]/div/div/div[2]/div[2]/div[{i}]/a/apec-recherche-resultat/div/div/div/div[2]/ul[2]/li[2]'
            lieu_element = driver.find_element(By.XPATH, lieu_xpath)
            lieu = lieu_element.text

            type_contrat_xpath = f'/html/body/main/div/apec-offres/div/apec-recherche-emploi/section/div[2]/div[3]/div/div/div[2]/div[2]/div[{i}]/a/apec-recherche-resultat/div/div/div/div[2]/ul[2]/li[1]'
            type_contrat_element = driver.find_element(By.XPATH, type_contrat_xpath)
            type_contrat = type_contrat_element.text

            date_publication_xpath = f'/html/body/main/div/apec-offres/div/apec-recherche-emploi/section/div[2]/div[3]/div/div/div[2]/div[2]/div[{i}]/a/apec-recherche-resultat/div/div/div/div[2]/ul[2]/li[3]'
            date_publication_element = driver.find_element(By.XPATH, date_publication_xpath)
            date_publication = date_publication_element.text

            # Cliquer sur l'offre
            offer_xpath = f'/html/body/main/div/apec-offres/div/apec-recherche-emploi/section/div[2]/div[3]/div/div/div[2]/div[2]/div[{i}]/a/apec-recherche-resultat/div/div'
            offer = driver.find_element(By.XPATH, offer_xpath)
            offer.click()

            # Attendre que les informations soient visibles
            title_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '/html/body/main/div/apec-offres/div/apec-detail-emploi/apec-header-nav/div/div/div/div/nav/div/div/h1'))
            )
            title = title_element.text

            description_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '/html/body/main/div/apec-offres/div/apec-detail-emploi/div/div/div[2]/article/div/div/apec-poste-informations/div/div[2]/div[1]/p[1]'))
            )
            description = description_element.text

            profil_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '/html/body/main/div/apec-offres/div/apec-detail-emploi/div/div/div[2]/article/div/div/apec-poste-informations/div/div[2]/div[1]/p[2]'))
            )
            profil = profil_element.text

            info_entreprise_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '/html/body/main/div/apec-offres/div/apec-detail-emploi/div/div/div[2]/article/div/div/apec-poste-informations/div/div[2]/div[1]/div[4]/p'))
            )
            info_entreprise = info_entreprise_element.text

            salaire_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '/html/body/main/div/apec-offres/div/apec-detail-emploi/div/div/div[2]/article/div/div/apec-poste-informations/div/div[1]/div[1]/span'))
            )
            salaire = salaire_element.text

            niveau_experience_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '/html/body/main/div/apec-offres/div/apec-detail-emploi/div/div/div[2]/article/div/div/apec-poste-informations/div/div[1]/div[3]/span'))
            )
            niveau_experience = niveau_experience_element.text

            statut_poste_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '/html/body/main/div/apec-offres/div/apec-detail-emploi/div/div/div[2]/article/div/div/apec-poste-informations/div/div[1]/div[5]/span'))
            )
            statut_poste = statut_poste_element.text

            secteur_activite_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '/html/body/main/div/apec-offres/div/apec-detail-emploi/div/div/div[2]/article/div/div/apec-poste-informations/div/div[1]/div[7]/span'))
            )
            secteur_activite = secteur_activite_element.text

            # Ajouter les informations à leurs listes respectives
            titles_list.append(title)
            description_list.append(description)
            profil_list.append(profil)
            info_entreprise_list.append(info_entreprise)
            type_contrat_list.append(type_contrat)
            lieu_list.append(lieu)
            date_publication_list.append(date_publication)
            salaire_list.append(salaire)
            niveau_experience_list.append(niveau_experience)
            statut_poste_list.append(statut_poste)
            secteur_activite_list.append(secteur_activite)
            entreprise_list.append(entreprise)

            # Revenir à la page précédente
            driver.back()

            # Attendre que la page précédente soit chargée
            driver.implicitly_wait(5)

        except Exception as e:
            print(f"Erreur lors du traitement de l'offre {i} sur la page {page}: {e}")

# Créer un DataFrame à partir des listes d'informations
df = pd.DataFrame({
    "Titres": titles_list, 
    "Descriptions": description_list, 
    "Profils": profil_list,
    "Info Entreprise": info_entreprise_list,
    "Type Contrat": type_contrat_list,
    "Lieu": lieu_list,
    "Date de publication": date_publication_list,
    "Salaire": salaire_list,
    "Niveau d'expérience": niveau_experience_list,
    "Statut du poste": statut_poste_list,
    "Secteur d'activité": secteur_activite_list,
    "Entreprise": entreprise_list
})

# Fermer le navigateur
driver.quit()

In [ ]:
# Filtrer les lignes contenant les mots clés dans la colonne "Titres" (environ 68 lignes)
df2= df[df['Titres'].str.contains(r'\b(data scientist|data science|datascience|datascientist)\b', case=False, na=False)]
df2.shape

In [35]:
#On fait une concaténation des data frame 'data_engineer_df' et data frame data scientist 'df2':
data_apec = pd.concat([data_engineer_df, df2], ignore_index=True)
data_apec.shape

In [40]:
#Transfomer variable 'Niveau d'experience' en 'experience Oui/Non' pour être comparable à data frame Pole emploi:
#'Minimum 2 ans', 'Minimum 5 ans', 'Minimum 3 ans','Minimum 6 ans','Minimum 1 an', 'Minimum 4 ans','Minimum 8 ans' --> OUI
#"Tous niveaux d'expérience acceptés",  'Aucune expérience exigée' --> NON
data_apec['experienceOuiNon'] = np.where(data_apec['Niveau d\'expérience'].isin(['Tous niveaux d\'expérience acceptés', 'Aucune expérience exigée']),'non', np.where(data_apec['Niveau d\'expérience'].isin(['Minimum 2 ans','Minimum 5 ans','Minimum 3 ans','Minimum 6 ans','Minimum 1 an','Minimum 4 ans','Minimum 8 ans']), 'oui', None))
data_apec= data_apec.drop(columns=['Niveau d\'expérience'])

In [47]:
#Transformer variable 'lieu' en 'ville' et 'departement':
# Division de la colonne "Lieu" en "Dept" et "Ville"
df_Dept_ville = data_apec["Lieu"].str.split(" - ", expand=True)
df_Dept_ville.columns = ["Ville", "Dept"]

# Création des nouvelles colonnes "Departement" et "Ville" dans le dataframe
data_apec["departement"] = df_Dept_ville["Dept"]
data_apec["ville"] = df_Dept_ville["Ville"]
data_apec = data_apec.drop(columns=['Lieu'])

In [55]:
#Changer les noms des variables:
data_apec= data_apec.rename(columns= {'Titres' : 'intitule', 'Descriptions':'description_offre', 'Profils': 'profils', 'Info Entreprise':'entreprise_description', 'Type Contrat': 'typeContrat', 'Date de publication':'dateCreation', 'Salaire': 'salaire', 'Statut du poste':'qualificationLibelle', 'Secteur d\'activité': 'secteurActiviteLibelle', 'Entreprise':'entreprise_nom'})

## Étape 2: Scrapping du site de Pôle Emploi

Étape A: Demander un 'access token' à Pôle Emploi Access Management en spécifiant les API souhaitées et l'id et secret: Pôle emploi access management génère et transmet un 'access token' valable pendant 24 minutes.

In [96]:
# URL pour obtenir le jeton d'accès
token_url = 'https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=%2Fpartenaire'

# Remplacez 'VOTRE_IDENTIFIANT_CLIENT' et 'VOTRE_CLE_SECRETE' par les valeurs réelles fournies par Pôle Emploi
client_id = 'PAR_poleemploijobsscrapin_4280e6316989cfda7bc759c90500ef347cc95780c6b1d5b67cca3db6c6d373d1'
client_secret = '70d252725959266d147615699b09d63746184cec836d7ff41f1bec6427215373'
# Scopes autorisés pour votre application
scopes = 'api_offresdemploiv2 o2dsoffre'  # Remplacez par les scopes valides pour votre application

# Corps de la requête avec ces paramètres
token_params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'scope': scopes
}

# En-tête de la requête
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

# Faire la requête pour obtenir le jeton d'accès
# Demander l'access token avec 'post'
response = requests.post(token_url, data=token_params, headers=headers)
access_token=response.json().get('access_token')

Étape B: requêter une API: on fourni l'access token à chaque interrogation d'une API

In [97]:
# Requêter l'API avec 'get':
# Soit on spécifie les paramètres dans l'url soit on les mets dans une liste 'params'
#url = 'https://api.pole-emploi.io/partenaire/offresdemploi/v2/offres/search?range=0-2&sort=1&motsCles=data%20engineer'
url = 'https://api.pole-emploi.io/partenaire/offresdemploi/v2/offres/search'

headers = {
    'Authorization': f'Bearer {access_token}'
}

#Boucle pour faire une requête pour chaque mot-clé:
mots_cles_list = ['data engineer', 'data scientist']
responses= []
for mots_cles in mots_cles_list:
    params = {
    'range': '0-149',
    'sort': '1',
    'motsCles': mots_cles,
    }

    # Faire la requête avec les en-têtes d'authentification
    reponse = requests.get(url, headers=headers, params=params)
    responses.append(reponse.json())

responses_json = json.dumps(responses)
responses_list = json.loads(responses_json)
resultats_list = [item['resultats'] for item in responses_list]
resultats_concat = [resultat for sublist in resultats_list for resultat in sublist]
df = pd.DataFrame(resultats_concat)

In [98]:
lieu_travail_df = pd.json_normalize(df['lieuTravail'])
df = pd.concat([df, lieu_travail_df], axis=1)
df = df.drop(columns=['lieuTravail'])

In [99]:
# Division de la colonne "libelle" en "Dept" et "Ville"
df_Dept_ville = df["libelle"].str.split(" - ", expand=True)
df_Dept_ville.columns = ["Dept", "Ville"]

# Création des nouvelles colonnes "Departement" et "Ville" dans le dataframe
df["departement"] = df_Dept_ville["Dept"]
df["ville"] = df_Dept_ville["Ville"]
df = df.drop(columns=['libelle'])

In [100]:
df['experienceOuiNon'] = np.where(df['experienceExige'] == 'D', 'non', np.where(df['experienceExige'].isin(['E', 'S']), 'oui', None))

In [101]:
df.columns.values[2] = 'description_offre'
entreprise_df = pd.json_normalize(df['entreprise'])
df = pd.concat([df, entreprise_df], axis=1)
df = df.drop(columns=['entreprise'])

In [102]:
df= df.rename(columns= {'description' : 'entreprise_description', 'nom':'entreprise_nom'})

In [103]:
#convertir le format de la date en 'DD/MM/YYYY':
df['dateCreation'] = pd.to_datetime(df['dateCreation'], utc=True)
df['formatted_date'] = df['dateCreation'].dt.strftime('%d/%m/%Y')
df['dateCreation'] = df['formatted_date']
df.drop(columns=['formatted_date'], inplace=True)

In [104]:
salaire_df = pd.json_normalize(df['salaire'])
df = pd.concat([df, salaire_df], axis=1)
df = df.drop(columns=['salaire'])
df= df.rename(columns= {'libelle' : 'salaire'})
df = df.drop(columns=['commentaire', 'complement1', 'complement2'])

In [105]:
#Garder que les colonnes nécessaires
colonnes_conserver = ['intitule', 'description_offre', 'dateCreation', 'typeContrat', 'experienceExige','experienceOuiNon','latitude', 'longitude', 'codePostal', 'commune','departement','ville', 'entreprise_description', 'entreprise_nom', 'salaire', 'secteurActiviteLibelle', 'qualificationLibelle']  
df = df.drop(columns=df.columns.difference(colonnes_conserver))

## Étape 3: merge des data frame "data_apec" (Apec) et "df" (Pole Emploi)

In [107]:
data_final = pd.merge(df, data_apec, left_on=['intitule', 'description_offre', 'dateCreation', 'typeContrat', 'experienceOuiNon', 'salaire', 'secteurActiviteLibelle', 'qualificationLibelle', 'departement', 'ville', 'entreprise_nom', 'entreprise_description'], right_on=['intitule', 'description_offre', 'dateCreation', 'typeContrat', 'experienceOuiNon', 'salaire', 'secteurActiviteLibelle', 'qualificationLibelle', 'departement', 'ville', 'entreprise_nom', 'entreprise_description'], how='outer')

In [117]:
data_final.to_csv('Jobs.csv', index=False)